In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from matplotlib import pyplot
import matplotlib.pyplot as plt 
from numpy import asarray
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from matplotlib import pyplot

https://machinelearningmastery.com/backtest-machine-learning-models-time-series-forecasting/

# Updating

In [ ]:
df_all = pd.read_csv('all_normalized.csv')

In [ ]:
def series_to_supervised(data, cols_list ,n_in=1, n_out=1, dropnan=True):
    x_cols = cols_list[:-1]
    y_cols = cols_list[-1]
    n_vars = 1 if type(data) is list else data.shape[1]
    new_vars = (n_vars - n_out)*n_in
    l = x_cols.copy()
    for i in range(1, n_in+1):
        new_l = []
        for j in range(len(x_cols)):
            new_l.append(x_cols[j] + "(t-" + str(i) + ")")
        l[0:0] = new_l
    df = DataFrame(data)
    y = df.iloc[:, -1].shift(-1)
    df = df.iloc[:, :-1]
    cols = list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
    # put it all together
    agg = concat(cols, axis=1)
    # drop rows with NaN values
    agg = concat([agg, y], axis = 1)
    if dropnan:
        agg.dropna(inplace=True)
    return agg.values, l




# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
    return data[:-n_test, :], data[-n_test:, :]

# fit an random forest model and make a one step prediction
def random_forest_forecast(train, testX, cols, detail):
    # transform list into array
    train = asarray(train)
    # split into input and output columns
    trainX, trainy = train[:-1, :-1], train[:-1, -1]
    # fit model
    n_feature = trainX.shape[0]
    feature_list = list(range(n_feature))
    model = RandomForestRegressor(n_estimators=1000, n_jobs=-1)
    model.fit(trainX, trainy)
    importances = list(model.feature_importances_)
    imp3 = list()
# List of tuples with variable and importance
    feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
    feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
    if detail:
        for i in range(0, 3):
            print((cols[feature_importances[i][0]], feature_importances[i][1]))
            imp3.append(cols[feature_importances[i][0]])
            
    #print(feature_importances[:3])
#     [print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]
    # make a one-step prediction
#     print("TrainX")
#     print(trainX.shape)
#     print("TrainY")
#     print(trainy)
#     print("TestX")
#     print(testX.shape)
#     print(testX)
    yhat = model.predict([testX])
    return yhat[0], imp3

# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cols, detail):
    predictions = list()
    imp_l = list()
    # split dataset
    train, test = train_test_split(data, n_test)
    # seed history with training dataset
    history = [x for x in train]
    # step over each time-step in the test set
    for i in range(len(test)):
        # split test row into input and output columns
        testX, testy = test[i, :-1], test[i, -1]
        # fit model on history and make a prediction
        # Not Anchored
        #yhat = random_forest_forecast(history[i:], testX)
        # Anchored
        yhat, imp3 = random_forest_forecast(history, testX, cols, detail)
        # store forecast in list of predictions
        predictions.append(yhat)
        
        for j in range(len(imp3)):
            imp_l.append(imp3[j])
        
        # add actual observation to history for the next loop
        history.append(test[i])
        # summarize progress
        if detail:
            print('>expected=%.1f, predicted=%.1f' % (testy, yhat), "\n")
    # estimate prediction error
#     error = mean_absolute_error(test[:, -1], predictions)
    errors = []
    for i in range(len(predictions)):
        errors.append(abs(test[i, -1] - predictions[i]) / test[i, -1])
    error = np.mean(errors)
    return error, test[:, -1], predictions, imp_l

#---------------------------------------------------------------------------------------

def data_cleaning(region = "United States", date_start = '2020-03-01', date_end = '2021-12-01'):
    df_all = pd.read_csv('all_normalized.csv') # read in the original file
    df = df_all[['country_name', 
                 'date', 
    #                  'stringency_index_x', 
    #                  'government_response_index', 
    #                  'containment_health_index', 
    #                  'economic_support_index', 
                 'c1_school_closing',
                 'c2_workplace_closing',
                 'c3_cancel_public_events',
                 'c4_restrictions_on_gatherings',
                 'c5_close_public_transport',
                 'c6_stay_at_home_requirements',
                 'c7_movementrestrictions',
                 'c8_internationaltravel',
                 'e1_income_support',
                 'e2_debtrelief',
                 'h1_public_information_campaigns',
                 'h2_testing_policy',
                 'h3_contact_tracing',
                 'h6_facial_coverings',
                 'h7_vaccination_policy',
                 'h8_protection_of_elderly_people',
                 'new_cases_smoothed_per_million']]

    df = df.loc[df['country_name'] == region]
    df.set_index("date",inplace=True)
    df.loc[date_start:date_end]
    df = df.dropna(axis=0)
    df = df.loc[date_start:date_end]
    df["date_count"] = list(range(df.shape[0]))


    # dummies=pd.concat((pd.get_dummies(df['country_name']), df),axis=1) # create dummies on country names
    # dummies.set_index("date",inplace=True) # set date as index
    # dummies.index=pd.to_datetime(dummies.index)
    # ds=dummies.index.to_series()
    # dummies['Month']=ds.dt.month # create dummies for date
    # dummies['Day']=ds.dt.day
    # dummies['Year']=ds.dt.year



    # dummies = dummies.loc[date_start:date_end]
    # dummies = dummies.drop('country_name', axis = 1)
    # df = dummies.dropna(axis=0)
    # df
    cols = df.columns.tolist()
    y = 'new_cases_smoothed_per_million'
    cols = cols[1:-2]
    cols.append('date_count')
    cols.append(y)
    df = df[cols]

    cols_list = df.columns.tolist()
    df_array = df.to_numpy()
    return df, df_array, cols_list
#---------------------------------------------------------------------------------------

# df = pd.DataFrame({'x1':[1,2,3,4,5, 6, 7, 8], 
#                    'x2':[10,20,40,50,60, 70, 80 , 90], 
#                    'y1': [1,4,9,16,25, 36, 49, 64]})
# print(df)

#---------------------------------------------------------------------------------------
# n_row = df.shape[0]
# cols_list = df.columns.tolist()
# df_array = df.to_numpy()

# df, df_array, cols_list = data_cleaning(region = "United States", date_start = '2020-03-01', date_end = '2021-12-01')

# data, col_names = series_to_supervised(df_array, cols_list, n_in=30 , n_out=1)
# # evaluate
# mre, y, yhat = walk_forward_validation(data, n_test = 30, cols = col_names)
# print('MRE: %.3f' % mre)

In [ ]:
def rf_model(region = "United States", 
             date_start = '2020-03-01', 
             date_end = '2021-12-01',
             n_in = 30, 
             n_out = 1, 
             n_test = 30,
             detail = True):
    df, df_array, cols_list = data_cleaning(region, date_start, date_end)
    data, col_names = series_to_supervised(df_array, cols_list, n_in , n_out)
    # evaluate
    mre, y, yhat, importance_variables = walk_forward_validation(data, n_test, col_names, detail)
    print('MRE: %.3f' % mre)
    plt.plot(y, label='Actual')
    plt.plot(yhat, 'r-', markersize=3, label='Prediction' )
    plt.ylabel('new_cases_smoothed_per_million')
    plt.xlabel("Test days")
    plt.title("RF in " + region)
    plt.legend()
    plt.show()
    frequencies = dict(zip(col_names, [0] * len(col_names)))
    for var in importance_variables:
        frequencies[var] += 1
    return frequencies

In [ ]:
regions = ['Taiwan', 
           'Croatia', 
           'Afghanistan', 
           'Iran', 
           'New Zealand', 
           'Macao', 
           'Estonia', 
           'China',
          'Belarus', 
           'Cambodia',
           'Russia', 
           'Senegal', 
           'Lithuania', 
           'Nigeria'
          ]


ith = 0

for region in regions:
    print("-----------------------------------------")
    print(region)
    dic1 = rf_model(region = region, 
             date_start = '2020-03-01', 
             date_end = '2021-12-01',
             n_in = 30, 
             n_out = 1, 
             n_test = 30,
        detail = True)
    
    if ith == 0:
        df_dic = pd.DataFrame.from_dict(dic1,  orient='index').T
        df_dic.index = [region]
    else:
        df_dic_temp = pd.DataFrame.from_dict(dic1,  orient='index').T
        df_dic_temp.index = [region]
        print(df_dic_temp)
        df_dic = df_dic.append(df_dic_temp)
    ith += 1


In [ ]:
df_temp = df_all[['country_name', 'continent']].groupby(['country_name', 'continent']).size().reset_index().rename(columns={0:'count'})
l1 = df_temp[['country_name']].values.reshape(1, -1).tolist()[0]
l2 = df_temp[['continent']].values.reshape(1, -1).tolist()[0]
dic_cc = dict(zip(l1, l2))

In [ ]:
final_df = df_dic.T[(df_dic.sum(axis = 0) != 0)].rename(dic_cc, axis='columns').T.groupby(level=0).sum().T

Only extracting top 10 importance Variables

In [ ]:
a = final_df.sum(axis =  1).sort_values(ascending = False)
l_name = a[:10].index.tolist()

In [ ]:
final_df.T[l_name].T.plot.bar(sort_columns = True, stacked = True)

If you want to see the all stacked bar graph

In [ ]:
df_dic.T[(df_dic.sum(axis = 0) != 0)].rename(dic_cc, axis='columns').T.groupby(level=0).sum().T.plot.bar(stacked = True)